In [143]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [5]:
nlp = spacy.load('en_core_web_sm')

In [40]:
sentence = "The burger was shit, however the food was alright"

In [41]:
doc = nlp(sentence)

In [42]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    children = [t.text for t in token.children]
    print(token.text, "\t", token.i, "\t", 
          token.pos_, "\t", token.dep_, "\t", 
          ancestors, "\t", children)

The 	 0 	 DET 	 det 	 ['burger', 'was', 'was'] 	 []
burger 	 1 	 NOUN 	 nsubj 	 ['was', 'was'] 	 ['The']
was 	 2 	 AUX 	 ccomp 	 ['was'] 	 ['burger', 'shit']
shit 	 3 	 ADJ 	 attr 	 ['was', 'was'] 	 []
, 	 4 	 PUNCT 	 punct 	 ['was'] 	 []
however 	 5 	 ADV 	 advmod 	 ['was'] 	 []
the 	 6 	 DET 	 det 	 ['food', 'was'] 	 []
food 	 7 	 NOUN 	 nsubj 	 ['was'] 	 ['the']
was 	 8 	 AUX 	 ROOT 	 [] 	 ['was', ',', 'however', 'food', 'alright']
alright 	 9 	 ADJ 	 acomp 	 ['was'] 	 []


In [43]:
def find_root_of_sentence(doc):
    root_token = None
    for token in doc:
        if (token.dep_ == "ROOT"):
            root_token = token
    return root_token

In [44]:
root_token = find_root_of_sentence(doc)


In [45]:
def find_other_verbs(doc, root_token):
    other_verbs = []
    for token in doc:
        ancestors = list(token.ancestors)
        if (token.pos_ == "VERB" and len(ancestors) == 1\
            and ancestors[0] == root_token):
            other_verbs.append(token)
    return other_verbs


In [46]:
other_verbs = find_other_verbs(doc, root_token)


In [47]:
def get_clause_token_span_for_verb(verb, doc, all_verbs):
    first_token_index = len(doc)
    last_token_index = 0
    this_verb_children = list(verb.children)
    for child in this_verb_children:
        if (child not in all_verbs):
            if (child.i < first_token_index):
                first_token_index = child.i
            if (child.i > last_token_index):
                last_token_index = child.i
    return(first_token_index, last_token_index)


In [48]:
token_spans = []   
all_verbs = [root_token] + other_verbs
for other_verb in all_verbs:
    (first_token_index, last_token_index) = \
     get_clause_token_span_for_verb(other_verb, 
                                    doc, all_verbs)
    token_spans.append((first_token_index, 
                        last_token_index))


In [49]:
sentence_clauses = []
for token_span in token_spans:
    start = token_span[0]
    end = token_span[1]
    if (start < end):
        clause = doc[start:end]
        sentence_clauses.append(clause)
sentence_clauses = sorted(sentence_clauses, 
                          key=lambda tup: tup[0])


In [50]:
clauses_text = [clause.text for clause in sentence_clauses]
print(clauses_text)

['was shit, however the food was']


In [233]:
import spacy
import deplacy
en = spacy.load('en_core_web_sm')
en.add_pipe('spacytextblob')

#Text 1 - With Punctuation and conj at each sentiment split
#Text 2 - With only 1 conjuction
#Text 3 - With no conj so we are only looking at the noun_dict

text = "In my opinion, the food was really really good , but their ambiance was shit, and my mom thinks the waiter was amazing"
# text = "In my opinion, the food was really really good but their ambiance was shit, my mom thinks the waiter was amazing"
# text = "In my opinion, the food was really really good, their ambiance was shit, my mom thinks the waiter was amazing"

doc = en(text)
deplacy.render(doc)

seen = set() # keep track of covered words

chunks = []
dict_noun = {} # This is for the dict of Noun - Adverb, adj, verb

for sent in doc.sents:
    #calculate the dict_noun first

    for word in sent:
            if word.pos_ == 'NOUN':
                current_noun = word.text
                dict_noun[current_noun] = []
            if word.pos_ == 'ADV' or word.pos_ == 'ADJ' or word.pos_ == 'VERB':
                dict_noun[current_noun].append(lol.text)

    #See if there are conjunctions to split on. If so get the heads for the subtrees
    heads = [cc for cc in sent if cc.dep_ == 'conj']
    print(heads)

    # If there are no conjunctions split only on punctuation.
    if(len(heads)==0):
        counter = 0
        array = []
        for word in sent:
            if word.pos_ != 'PUNCT':
                array.append(word)
                continue
            chunk = (' '.join([ww.text for ww in array]))
            chunks.append( (counter, chunk) )
            counter = counter + 1
            array = []
        chunk = (' '.join([ww.text for ww in array]))
        chunks.append( (counter, chunk) )
    else:
        for head in heads:
            words = [None]*100
            counter = 0
            for i in head.subtree:
                if(i in heads):
                    if(i != head):
                        break
                if(i.pos_ == 'PUNCT'):
                    break
                words[counter] = i
                counter = counter + 1
                if(i.dep_ == 'cc'):
                    break
            res = []
            for val in words:
                if val != None :
                    res.append(val)
            for word in res:
                seen.add(word)
            chunk = (' '.join([ww.text for ww in res]))
            chunks.append( (head.i, chunk) )
        unseen = [ww for ww in sent if ww not in seen]
        counter = 0
        array = []
        for word in unseen:
            if word.pos_ != 'PUNCT':
                array.append(word)
                continue
            chunk = (' '.join([ww.text for ww in array if ww.dep_ != 'cc']))
            chunks.append( (counter, chunk) )
            counter = counter + 1
            array = []
        chunk = (' '.join([ww.text for ww in array if ww.dep_ != 'cc']))
        chunks.append( (counter, chunk) )
#         chunk = ' '.join([ww.text for ww in unseen if ww.dep_ != 'cc' and ww.pos_ != 'PUNCT'])
#         chunks.append( (sent.root.i, chunk) )

chunks = sorted(chunks, key=lambda x: x[0])
print(dict_noun)
for ii, chunk in chunks:
    if (len(chunk) != 0):
        print("The chunk is: " + chunk)
    doc = en(chunk)
    for i in doc.sents:
        for cc in i:
            if cc.dep_ == 'nsubj':
                continue
                print("Noun :" + cc.text)
    if(doc._.polarity != 0.0):
        print("Polarity: " + str(doc._.polarity))

In       ADP   ═══╗<════╗       prep
my       PRON  <╗ ║     ║       poss
opinion  NOUN  ═╝<╝     ║       pobj
,        PUNCT <════════║═╗     punct
the      DET   <╗       ║ ║     det
food     NOUN  ═╝<════╗ ║ ║     nsubj
was      AUX   ═╗═╗═╗═╝═╝═╝═╗═╗ ROOT
really   ADV   <╝ ║ ║       ║ ║ advmod
really   ADV   <╗ ║ ║       ║ ║ advmod
good     ADJ   ═╝<╝ ║       ║ ║ acomp
,        PUNCT <════║═══════╝ ║ punct
but      CCONJ <════╝         ║ cc
their    PRON  <╗             ║ poss
ambiance NOUN  ═╝<══╗         ║ nsubj
was      VERB  ═╗═╗═╝═╗═╗<════╝ conj
shit     VERB  <╝ ║   ║ ║       acomp
,        PUNCT <══║═══╝ ║       punct
and      CCONJ <══╝     ║       cc
my       PRON  <╗       ║       poss
mom      NOUN  ═╝<════╗ ║       nsubj
thinks   VERB  ═════╗═╝<╝       conj
the      DET   <╗   ║           det
waiter   NOUN  ═╝<╗ ║           nsubj
was      AUX   ═╗═╝<╝           ccomp
amazing  ADJ   <╝               acomp
[was, thinks]
{'opinion': [], 'food': ['amazing', 'amazing', 'amaz